In [3]:
import json
import gzip
import pyodbc
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import text
import urllib
from urllib.request import urlopen
import traceback
import logging
from azure.storage.blob import BlobServiceClient
import time
import pandas as pd
import os

In [4]:
#Connect to database
def connect_to_azure():
   
    params = urllib.parse.quote_plus(r'Driver={ODBC Driver 17 for SQL Server};Server=#DB_URL#;Database=#DBNAME#;Uid=#DB_ID#;Pwd=#DB_KEY#;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')

    conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
    engine_azure = create_engine(conn_str,echo=False)

    print('connection is ok')
    print(engine_azure.table_names())

    return engine_azure

In [5]:
engine_azure = connect_to_azure()

connection is ok
['all_pubs_linked', 'cord_publications', 'history', 'join_all_pubs_projs', 'join_oa_cord_doi', 'join_src_pubs_rels', 'join_trg_pubs_rels', 'oa_projects', 'oa_publications', 'oa_publications_three', 'oa_publications_three_copy', 'oa_publications_two', 'oa_pubs', 'oa_relationships', 'oa_relationships_copy', 'oa_relationships_copy_two', 'projects', 'projects_two']


## Step 1. Get OpenAIRE publications that are in CORD19 and related to funded projects

In [6]:
df_final = pd.read_sql_table('join_all_pubs_projs', engine_azure)
df_final.head()

,oaid,oa_title,oa_doi,oa_pmid,oa_arxiv,oa_pub_date,cord_uid,cord_title,cord_doi,cord_pmcid,...,proj_oaid,proj_code,proj_title,proj_startdate,proj_enddate,currency,amount,jurisdiction,longname,shortname
0,50|dedup_wf_001::9001ec12701e4ce5a66adbd25a377437,GROMACS: High performance molecular simulation...,10.1080/07391102.2020.1824816,empty,empty,empty,h62ii7ir,Repurposing of the approved small molecule dru...,10.1080/07391102.2020.1824816,PMC7576931,...,40|corda_______::b21f6b1618151028fe7e3b4a2f2f894f,258980,Million-core Molecular Simulation,2011-05-01,2017-04-30,empty,0.0,EU;,European Commission;,EC;
1,50|dedup_wf_001::7667e9e8013e45658fb3eae0be158d10,Imaging and spectroscopy of domains of the cel...,10.1039/d0an00696c,empty,empty,empty,fhr40sr3,Imaging and spectroscopy of domains of the cel...,10.1039/d0an00696c,None,...,40|corda__h2020::0875aa41d7225523933cecf91d83e5ab,665778,SUPPORTING MOBILITY IN THE ERA THROUGH AN INTE...,2015-09-01,2021-05-31,EUR,5841000.0,EU;,European Commission;,EC;
2,50|dedup_wf_001::e1d688bcaa8d8ff6760a1031106e130f,Subsumption Demodulation in First-Order Theore...,10.1007/978-3-030-51074-9_17,,empty,empty,pjnxv3zg,Subsumption Demodulation in First-Order Theore...,10.1007/978-3-030-51074-9_17,PMC7324223,...,40|fwf_________::91dbeb8c3d808e456bbe8df2403038ee,W 1255,Vollantrag zu Logical Methods in Computer Science,2014-03-01,2022-02-28,EUR,6385130.0,AT;,Austrian Science Fund (FWF);,FWF;
3,50|dedup_wf_001::7521e47071fa7dc2223634cf64d07fe3,Induction of Cell-Cell Fusion by Ebola Virus G...,10.1371/journal.ppat.1005373,26730950,empty,empty,7ycgd0h7,Induction of Cell-Cell Fusion by Ebola Virus G...,10.1371/journal.ppat.1005373,PMC4711667,...,40|nih_________::01565c59f11b660d408e6da3ca911200,5R01AI053668-14,Entry mechanisms used by a model retrovirus,2003-01-01,2018-04-30,empty,0.0,US;,National Institutes of Health;,NIH;
4,50|dedup_wf_001::2c17496023455b7cad411efe9b5e1cdc,Interferon-induced transmembrane protein 3 blo...,10.1371/journal.ppat.1007532,30640957,empty,empty,15wxk8lt,Interferon-induced transmembrane protein 3 blo...,10.1371/journal.ppat.1007532,PMC6347298,...,40|nih_________::01565c59f11b660d408e6da3ca911200,5R01AI053668-14,Entry mechanisms used by a model retrovirus,2003-01-01,2018-04-30,empty,0.0,US;,National Institutes of Health;,NIH;


In [7]:
#Count projects per funder
proj_by_funders = df_final.groupby('longname')['proj_oaid'].nunique()
proj_by_funders.head()

longname
Academy of Finland;                        46
Australian Research Council (ARC);         44
Austrian Science Fund (FWF);               34
Canadian Institutes of Health Research;     1
Croatian Science Foundation (CSF);          3
Name: proj_oaid, dtype: int64

In [8]:
#Count CORD publications by funders
cov_pubs_by_funders = df_final.groupby("longname")['oaid'].count()
cov_pubs_by_funders.head()

longname
Academy of Finland;                         85
Australian Research Council (ARC);          50
Austrian Science Fund (FWF);                53
Canadian Institutes of Health Research;    788
Croatian Science Foundation (CSF);           7
Name: oaid, dtype: int64

In [9]:
#Calculate the funded amount
pd.options.display.float_format = '{:.2f}'.format
amount_by_funders = df_final.groupby("longname", sort = True)['amount'].sum()

In [10]:
#Aggregate data by funders
new_df = pd.DataFrame()
new_df['funder'] = proj_by_funders.index
new_df['COVID projects'] = proj_by_funders.values
new_df['COVID publications'] = cov_pubs_by_funders.values
new_df['project to pub ratio'] = new_df['COVID publications'] / new_df['COVID projects']
new_df['amount'] = amount_by_funders.values
new_df['amount to pub ratio'] = new_df['amount'] / new_df['COVID publications']
new_df

,funder,COVID projects,COVID publications,project to pub ratio,amount,amount to pub ratio
0,Academy of Finland;,46,85,1.85,37316502.00,439017.67
1,Australian Research Council (ARC);,44,50,1.14,0.00,0.00
2,Austrian Science Fund (FWF);,34,53,1.56,94655566.00,1785954.08
3,Canadian Institutes of Health Research;,1,788,788.00,0.00,0.00
4,Croatian Science Foundation (CSF);,3,7,2.33,0.00,0.00
5,European Commission;,657,1814,2.76,6050751891.80,3335585.39
6,"Fundação para a Ciência e a Tecnologia, I.P.;",48,83,1.73,0.00,0.00
7,"Ministry of Education, Science and Technologic...",20,31,1.55,0.00,0.00
8,"Ministry of Science, Education and Sports of t...",4,9,2.25,0.00,0.00
9,National Health and Medical Research Council (...,192,398,2.07,0.00,0.00


In [11]:
#Drop duplicates
df_final = df_final.drop_duplicates(subset=['oaid'])

## Step 2. Use Semantic Scholar API to obtain citations/influential citations

In [20]:
#Given url, access semanticscholar api and get data
#returns dictionary of records
def get_sem_data(url):
    result = {}
    
    try:
        # store the response of URL
        response = urlopen(url)
        if response.status == 403:
            alert('Rate limited. Waiting to retry…')
            wait(response.retry-after)
            retry(url)
        
        # storing the JSON response 
        # from url in data
        data_json = json.loads(response.read())
        
        result['citations'] = str(data_json['numCitedBy'])
        result['influential citations'] = str(data_json['influentialCitationCount'])
        
        return result
    except Exception as e:
        if e.code == 404:
            pass
        else:
            logging.error(traceback.format_exc())

In [ ]:
#Construct url and get semantic scholar data, saving it to the local folder
def get_semantic_scholar_citations():
    base_url = 'https://api.semanticscholar.org/v1/paper/'
    sem_list = []
    counter = 0
    start = 0
    for index, row in df_final.iterrows():
        if counter >= start:
            end_url = base_url + row['oa_doi']
            try:
                res_obj = get_sem_data(end_url)

                res_obj['oaid'] = row['oa_doi']

                sem_list.append(res_obj)
            except:
                print('problems with link')
            
            #We need to wait after each 100 requests due to API limitations
            if counter % 100 == 0:
                df_sem = pd.DataFrame(sem_list)
                filename = 'semantic_ + ' + str(start) + '_' + str(counter) + '.csv'
                df_sem.to_csv(filename,index=False)
                time.sleep(302)
            df_sem = pd.DataFrame(sem_list)
            filename = 'semantic_ + ' + str(start) + '_' + str(counter) + '.csv'
            df_sem.to_csv(filename,index=False)
        counter += 1

In [17]:
def get_citation_table():
    filenames = os.listdir()
    semantic_files = [filename for filename in filenames if filename.startswith('semantic_')]
    
    dfs = [pd.read_csv(filename) for filename in semantic_files]
    final_df = pd.concat(dfs, ignore_index=True)
    
    df_sum_unique = final_df.drop_duplicates()

    return df_sum_unique

In [18]:
#First, files from semantic_data have to be unpacked into the same folder as the notebook
citation_table = get_citation_table()
citation_table.head()

,citations,influential citations,oaid
0,6,0,10.1080/07391102.2020.1824816
1,0,0,10.1039/d0an00696c
2,4,0,10.1007/978-3-030-51074-9_17
3,30,2,10.1371/journal.ppat.1005373
4,34,7,10.1371/journal.ppat.1007532


## Step 3. Enrich final table with citations and influential citations 

In [20]:
df = pd.merge(df_final,citation_table,left_on='oa_doi', right_on='oaid',how='left')

In [21]:
#Export to excel
df.to_excel('aLL_rows_table.xlsx')

### Extract data for European Commission

In [22]:
ecdf = df.loc[df['shortname'] == 'EC;']
ecdf

,oaid_x,oa_title,oa_doi,oa_pmid,oa_arxiv,oa_pub_date,cord_uid,cord_title,cord_doi,cord_pmcid,...,proj_startdate,proj_enddate,currency,amount,jurisdiction,longname,shortname,citations,influential citations,oaid_y
0,50|dedup_wf_001::9001ec12701e4ce5a66adbd25a377437,GROMACS: High performance molecular simulation...,10.1080/07391102.2020.1824816,empty,empty,empty,h62ii7ir,Repurposing of the approved small molecule dru...,10.1080/07391102.2020.1824816,PMC7576931,...,2011-05-01,2017-04-30,empty,0.00,EU;,European Commission;,EC;,6.00,0.00,10.1080/07391102.2020.1824816
1,50|dedup_wf_001::7667e9e8013e45658fb3eae0be158d10,Imaging and spectroscopy of domains of the cel...,10.1039/d0an00696c,empty,empty,empty,fhr40sr3,Imaging and spectroscopy of domains of the cel...,10.1039/d0an00696c,None,...,2015-09-01,2021-05-31,EUR,5841000.00,EU;,European Commission;,EC;,0.00,0.00,10.1039/d0an00696c
35,50|dedup_wf_001::ba6d49008ea5b759d7842bab41ef100c,Treatment of COVID-19 Pneumonia: the Case for ...,10.1007/s12015-020-10004-x,32696426,empty,empty,9j9ni3wg,Treatment of COVID-19 Pneumonia: the Case for ...,10.1007/s12015-020-10004-x,PMC7372209,...,2020-01-01,2024-12-31,EUR,6844620.00,EU;,European Commission;,EC;,2.00,0.00,10.1007/s12015-020-10004-x
36,50|dedup_wf_001::545751d3799352206b93e1a4f2b34b4a,Histo-blood group glycans in the context of pe...,10.1016/j.bbagen.2015.12.026,26748235,empty,empty,5p9j5xj0,Histo-blood group glycans in the context of pe...,10.1016/j.bbagen.2015.12.026,PMC7117023,...,2015-09-01,2019-08-31,EUR,3293890.00,EU;,European Commission;,EC;,18.00,1.00,10.1016/j.bbagen.2015.12.026
37,50|dedup_wf_001::4e9ab37f254220b9f1f1950283065f58,How to organize an online conference,10.1038/s41578-020-0194-0,empty,empty,empty,hg4edj36,How to organize an online conference,10.1038/s41578-020-0194-0,PMC7095294,...,2017-04-01,2022-09-30,EUR,1999890.00,EU;,European Commission;,EC;,20.00,0.00,10.1038/s41578-020-0194-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5410,50|dedup_wf_001::0aa743d2ff4a7f111c7381e39cd7579d,Characterization of a Novel RNA Virus Discover...,10.3390/v9080214,,empty,empty,1baso3q2,Characterization of a Novel RNA Virus Discover...,10.3390/v9080214,PMC5580471,...,2011-01-01,2015-12-31,empty,0.00,EU;,European Commission;,EC;,6.00,0.00,10.3390/v9080214
5411,50|dedup_wf_001::bf94c29c77af374502de61584b6f76ec,Predicting the effects of parasite co-infectio...,10.1098/rspb.2017.2610,29540516,empty,empty,vcrbzzhu,Predicting the effects of parasite co-infectio...,10.1098/rspb.2017.2610,PMC5879626,...,2015-05-22,2016-08-21,EUR,112673.00,EU;,European Commission;,EC;,14.00,0.00,10.1098/rspb.2017.2610
5423,50|dedup_wf_001::483a90ed604f97e529e467abba4ad91f,Harnessing publicly available genetic data to ...,10.1007/s00439-016-1647-9,26946290,empty,empty,wrwp7ogi,Harnessing publicly available genetic data to ...,10.1007/s00439-016-1647-9,PMC4835528,...,2013-10-01,2017-03-31,empty,0.00,EU;,European Commission;,EC;,10.00,2.00,10.1007/s00439-016-1647-9
5424,50|dedup_wf_001::967b21879fc0fb82d8c255296472ecba,Comparison of phenotypic and genotypic diagnos...,10.1016/j.jcv.2019.09.003,31521013,empty,empty,awr2tbj0,Comparison of phenotypic and genotypic diagnos...,10.1016/j.jcv.2019.09.003,PMC7106360,...,2013-09-01,2017-02-28,empty,0.00,EU;,European Commission;,EC;,1.00,0.00,10.1016/j.jcv.2019.09.003


### Find top projects by citations

In [23]:
ecdf.groupby(['proj_title'])['citations'].agg('sum').reset_index().sort_values(by=['citations'], ascending=False).to_excel('top_proj.xlsx')

### Find top projects by influential citations

In [24]:
ecdf.groupby(['proj_title'])['influential citations'].agg('sum').reset_index().sort_values(by=['influential citations'], ascending=False).to_excel('top_proj_inf.xlsx')

### Find top projects by publication count

In [25]:
ecdf.groupby(['proj_title'])['citations'].agg('count').reset_index().sort_values(by=['citations'], ascending=False).to_excel('top_proj_pub_count.xlsx')

## Step 4. Aggregate the final table by funders

In [77]:
df.loc[df['citations'] > 0]

,oaid_x,oa_title,oa_doi,oa_pmid,oa_arxiv,oa_pub_date,cord_uid,cord_title,cord_doi,cord_pmcid,...,proj_startdate,proj_enddate,currency,amount,jurisdiction,longname,shortname,citations,influential citations,oaid_y
101,50|dedup_wf_001::3c8cd854535bbf53a45363e6fae9afd7,Efficient Sensing of Infected Cells in Absence...,10.1371/journal.ppat.1003412,23785283,empty,empty,2mceonc1,Efficient Sensing of Infected Cells in Absence...,10.1371/journal.ppat.1003412,PMC3681750,...,2010-04-01,2014-03-31,empty,0.00,EU;,European Commission;,EC;,41.00,2.00,10.1371/journal.ppat.1003412
102,50|dedup_wf_001::35cb19f220f7d3974726f3ce2a52df5e,COVID-19 pandemic changes the food consumption...,10.1016/j.tifs.2020.08.017,empty,empty,empty,l5r1hl8x,COVID-19 pandemic changes the food consumption...,10.1016/j.tifs.2020.08.017,PMC7462788,...,2019-10-01,2023-09-30,EUR,10189600.00,EU;,European Commission;,EC;,11.00,0.00,10.1016/j.tifs.2020.08.017
103,50|dedup_wf_001::c7adc248fd2b293c1b5de76ddd918896,Social network-based distancing strategies to ...,10.1038/s41562-020-0898-6,17283616,empty,empty,qpzg8lam,Social network-based distancing strategies to ...,10.1038/s41562-020-0898-6,None,...,2019-11-01,2024-10-31,EUR,2499810.00,EU;,European Commission;,EC;,174.00,10.00,10.1038/s41562-020-0898-6
104,50|dedup_wf_001::c0dc5cb4e9c24974eea633931eb9d797,Demographic science aids in understanding the ...,10.1101/2020.03.15.20036293,32400861,empty,empty,gv8wlo06,Demographic science aids in understanding the ...,10.1101/2020.03.15.20036293,None,...,2019-11-01,2024-10-31,EUR,2499810.00,EU;,European Commission;,EC;,13.00,0.00,10.1101/2020.03.15.20036293
105,50|dedup_wf_001::1568062058ffd299a162b5d4ac08510a,"Forecasting spatial, socioeconomic and demogra...",10.1186/s12916-020-01646-2,32594909,empty,empty,5nnlyavp,"Forecasting spatial, socioeconomic and demogra...",10.1186/s12916-020-01646-2,PMC7321716,...,2019-11-01,2024-10-31,EUR,2499810.00,EU;,European Commission;,EC;,13.00,0.00,10.1186/s12916-020-01646-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,50|dedup_wf_001::b0fdbfba077427c97120a881845d4780,Mesenchymal Stem Cell Infusion Shows Promise f...,10.14336/ad.2020.0301,32257554,empty,empty,v73bsrwg,Mesenchymal Stem Cell Infusion Shows Promise f...,10.14336/ad.2020.0301,PMC7069463,...,2018-06-15,2023-05-31,empty,0.00,US;,National Institutes of Health;,NIH;,61.00,5.00,10.14336/ad.2020.0301
5430,50|dedup_wf_001::829344a03f04c2a1a9445658316d15f8,Myocardial T1 mapping and extracellular volume...,10.1186/1532-429X-15-92,24124732,empty,empty,ogv0yga9,Myocardial T1 mapping and extracellular volume...,10.1186/1532-429x-15-92,None,...,empty,empty,empty,0.00,US;,National Institutes of Health;,NIH;,735.00,26.00,10.1186/1532-429X-15-92
5431,50|dedup_wf_001::de458fac816db7268d5ec90fca9e590b,Zika and Flavivirus Shell Disorder: Virulence ...,10.3390/biom9110710,31698857,empty,empty,nmt221tu,Zika and Flavivirus Shell Disorder: Virulence ...,10.3390/biom9110710,PMC6920988,...,2010-08-01,2021-07-31,empty,0.00,US;,National Science Foundation;,NSF;,13.00,0.00,10.3390/biom9110710
5432,50|dedup_wf_001::1cd6834b6cdd391dc04b4d2137e322bb,HIV Vaccine Mystery and Viral Shell Disorder,10.3390/biom9050178,31072073,empty,empty,q28h5zi8,HIV Vaccine Mystery and Viral Shell Disorder,10.3390/biom9050178,PMC6572542,...,2010-08-01,2021-07-31,empty,0.00,US;,National Science Foundation;,NSF;,19.00,2.00,10.3390/biom9050178


### Count projects per funder

In [62]:
proj_by_funders = df.groupby('longname')['proj_oaid'].nunique()
proj_by_funders.head()

longname
Academy of Finland;                        31
Australian Research Council (ARC);         38
Austrian Science Fund (FWF);               26
Canadian Institutes of Health Research;     1
Croatian Science Foundation (CSF);          3
Name: proj_oaid, dtype: int64

### Count CORD publications by funders

In [26]:
cov_pubs_by_funders = df.groupby("longname")['oaid_x'].count()
cov_pubs_by_funders.head()

longname
Academy of Finland;                         39
Australian Research Council (ARC);          42
Austrian Science Fund (FWF);                27
Canadian Institutes of Health Research;    472
Croatian Science Foundation (CSF);           4
Name: oaid_x, dtype: int64

In [27]:
pd.options.display.float_format = '{:.2f}'.format
amount_by_funders = df.groupby("longname", sort = True)['amount'].sum()

### Count citations and influential citations by funders

In [28]:
citations_by_funders = df.groupby("longname", sort = True)['citations'].sum()
citations_by_funders

longname
Academy of Finland;                                                                      952.00
Australian Research Council (ARC);                                                       502.00
Austrian Science Fund (FWF);                                                            1144.00
Canadian Institutes of Health Research;                                                24166.00
Croatian Science Foundation (CSF);                                                        67.00
European Commission;                                                                   39592.00
Fundação para a Ciência e a Tecnologia, I.P.;                                           2176.00
Ministry of Education, Science and Technological Development of Republic of Serbia;      244.00
Ministry of Science, Education and Sports of the Republic of Croatia (MSES);             108.00
National Health and Medical Research Council (NHMRC);                                   4103.00
National Institutes of Health; 

In [66]:
inf_citations_by_funders = df.groupby("longname", sort = True)['influential citations'].sum()
inf_citations_by_funders

longname
Academy of Finland;                                                                     45.00
Australian Research Council (ARC);                                                      22.00
Austrian Science Fund (FWF);                                                            48.00
Canadian Institutes of Health Research;                                                997.00
Croatian Science Foundation (CSF);                                                       3.00
European Commission;                                                                  1996.00
Fundação para a Ciência e a Tecnologia, I.P.;                                           89.00
Ministry of Education, Science and Technological Development of Republic of Serbia;     12.00
Ministry of Science, Education and Sports of the Republic of Croatia (MSES);             2.00
National Health and Medical Research Council (NHMRC);                                  164.00
National Institutes of Health;                     

### Construct the final dataframe for export to Excel

In [67]:
new_df = pd.DataFrame()
new_df['funder'] = proj_by_funders.index
new_df['COVID projects'] = proj_by_funders.values
new_df['COVID publications'] = cov_pubs_by_funders.values
new_df['project to pub ratio'] = new_df['COVID publications'] / new_df['COVID projects']
new_df['funding'] = amount_by_funders.values
new_df['funding to pub ratio'] = new_df['funding'] / new_df['COVID publications']
new_df['citations'] = citations_by_funders.values
new_df['influential citations'] = inf_citations_by_funders.values
new_df['citations to pub ratio'] = new_df['citations'] / new_df['COVID publications']
new_df['influential citations to pub ratio'] = new_df['influential citations'] / new_df['COVID publications']
new_df['funding to citations ratio'] = new_df['funding'] / new_df['citations']
new_df['funding to influential citations ratio'] = new_df['funding'] / new_df['influential citations']
new_df

,funder,COVID projects,COVID publications,project to pub ratio,funding,funding to pub ratio,citations,influential citations,citations to pub ratio,influential citations to pub ratio,funding to citations ratio,funding to influential citations ratio
0,Academy of Finland;,31,39,1.26,17766222.00,455544.15,735.00,45.00,18.85,1.15,24171.73,394804.93
1,Australian Research Council (ARC);,38,42,1.11,0.00,0.00,475.00,22.00,11.31,0.52,0.00,0.00
2,Austrian Science Fund (FWF);,26,27,1.04,39467117.00,1461745.07,1140.00,48.00,42.22,1.78,34620.28,822231.60
3,Canadian Institutes of Health Research;,1,472,472.00,0.00,0.00,24166.00,997.00,51.20,2.11,0.00,0.00
4,Croatian Science Foundation (CSF);,3,4,1.33,0.00,0.00,67.00,3.00,16.75,0.75,0.00,0.00
5,European Commission;,543,894,1.65,2947962098.40,3297496.75,39019.00,1996.00,43.65,2.23,75551.96,1476934.92
6,"Fundação para a Ciência e a Tecnologia, I.P.;",28,30,1.07,0.00,0.00,2176.00,89.00,72.53,2.97,0.00,0.00
7,"Ministry of Education, Science and Technologic...",12,13,1.08,0.00,0.00,244.00,12.00,18.77,0.92,0.00,0.00
8,"Ministry of Science, Education and Sports of t...",3,4,1.33,0.00,0.00,108.00,2.00,27.00,0.50,0.00,0.00
9,National Health and Medical Research Council (...,120,143,1.19,0.00,0.00,4048.00,164.00,28.31,1.15,0.00,0.00


### Export to Excel

In [ ]:
new_df.to_excel('final_table.xlsx')